# COSC2789-Practical Data Science
# Assignment 2: Data Modelling
### Due date: Jan 4, 2020
### Lecturer: Vo Ngoc Yen Nhi


###

### Student: Nguyen Quang Linh
### ID: s3697110


###

#### This notebook is the submission for the assignment 2. 

This notebook is going to explore the data of books as indexed on Goodreads. From this, models can be builed to predict the average rating of these books. 

#### Acknowlegdements  
Books are scraped from a list titles the "Best Books Ever" which can be found here https://www.goodreads.com/list/show/1.Best_Books_Ever page. We thank Kaggle Community for providing this dataset and Goodreads for housing the data.


# Import Libraries

In [1]:
#import necessary libraries
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

# seaborn theme
sns.set_theme(style="darkgrid")

In [2]:
# natural langue process libraries
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\quang\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\quang\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\quang\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
# max column
pd.options.display.max_columns = None

# Task 1.1: Data Prepatation

### Quick look on training dataset

In [4]:
# import training data.
X = pd.read_csv("X_train.csv")
#print data
X.head()

,id,title,link,series,cover_link,author,author_link,rating_count,review_count,number_of_pages,date_published,publisher,original_title,genre_and_votes,isbn,isbn13,asin,settings,characters,awards,amazon_redirect_link,worldcat_redirect_link,recommended_books,books_in_series,description
0,2533958,Frost/Nixon,https://www.goodreads.com//book/show/2533958.F...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,David Frost,https://www.goodreads.com/author/show/264173.D...,523,51,338.0,December 31st 1975,MacMillan,Frost/Nixon: Behind the Scenes of the Nixon In...,"History 42, Nonfiction 33, Politics 28, Biogra...",0230531148,9780230531147,NaN,NaN,"Richard Nixon, David Frost",NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"13167112, 13167112, 29429295, 2074322, 3025937...",NaN,Including hilarious tales of the people he enc...
1,16071790,Just What Kind of Mother Are You?,https://www.goodreads.com//book/show/16071790-...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,Paula Daly,https://www.goodreads.com/author/show/6175529....,10092,1236,320.0,April 28th 2013,Doubleday Canada,Just What Kind of Mother Are You?,"Mystery 136, Fiction 122, Thriller 90, Suspens...",0385680074,9780385680073,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/1?...,https://www.goodreads.com//book_link/follow/8?...,"42614702, 42614702, 42182331, 45730306, 379129...",NaN,A searing and sinister thriller for readers wh...
2,13825165,Cafenova,https://www.goodreads.com//book/show/13825165-...,(Clairmont #1),https://i.gr-assets.com/images/S/compressed.ph...,S. Jane Scheyder,https://www.goodreads.com/author/show/5871778....,769,90,318.0,November 17th 2010,Andres & Blanton,NaN,"Romance 52, Womens Fiction-Chick Lit 21, Chris...",0983031800,9780983031802,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"89959, 89959, 59091, 1996085, 25843420, 364288...",23498673,"Leaving her broken heart behind in Seattle, Ma..."
3,18061,Uncle Fred in the Springtime,https://www.goodreads.com//book/show/18061.Unc...,(Uncle Fred #1),https://i.gr-assets.com/images/S/compressed.ph...,P.G. Wodehouse,https://www.goodreads.com/author/show/7963.P_G...,4200,328,288.0,1939,Everyman,Uncle Fred in the Springtime,"Fiction 153, Humor 133, Humor-Comedy 31, Class...",1841591300,9781841591308,NaN,NaN,"Clarence Threepwood, Sebastian Beach, Rupert B...",NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"17910171, 17910171, 13418137, 48839090, 390885...","16241184, 16241186, 16241198",Pongo Twistleton is in a state of financial em...
4,3766428,Double Share,https://www.goodreads.com//book/show/3766428-d...,(Golden Age of the Solar Clipper #4),https://i.gr-assets.com/images/S/compressed.ph...,Nathan Lowell,https://www.goodreads.com/author/show/1048167....,4212,184,308.0,2008,Podiobooks,"Double Share (Golden Age of the Solar Clipper,...","Science Fiction 120, Space-Space Opera 37, Fic...",NaN,NaN,NaN,NaN,Ishmael Horatio Wang,Parsec Award Nominee for Best Speculative Fict...,https://www.goodreads.com//book_link/follow/17...,NaN,"53172544, 53172544, 20951387, 52856493, 444219...","2334538, 2906048, 4266110, 6963764, 10054649, ...","In his first assignment as an officer, Ishmael..."


### Quick look on testing data

In [5]:
# import testing data.
X_test = pd.read_csv("X_test.csv")
X_test.head()

,id,title,link,series,cover_link,author,author_link,rating_count,review_count,number_of_pages,date_published,publisher,original_title,genre_and_votes,isbn,isbn13,asin,settings,characters,awards,amazon_redirect_link,worldcat_redirect_link,recommended_books,books_in_series,description
0,6333213,The Way of the Bow,https://www.goodreads.com//book/show/6333213-t...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,Paulo Coelho,https://www.goodreads.com/author/show/566.Paul...,2179,180,76.0,2003,NaN,The Way of the Bow,"Fiction 39, Philosophy 15, Inspirational 6, Se...",0557010071,9.780557e+12,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"52049749, 52049749, 9242583, 50058600, 1194740...",NaN,âThe Way of the Bowâ relates the story of ...
1,493553,The Silver Child,https://www.goodreads.com//book/show/493553.Th...,(Silver Sequence #1),https://i.gr-assets.com/images/S/compressed.ph...,Cliff McNish,https://www.goodreads.com/author/show/27813.Cl...,874,84,192.0,August 1st 2003,First Avenue Editions,The Silver Child,"Fantasy 30, Young Adult 15, Science Fiction 14...",082256503X,9.780823e+12,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"13542616, 13542616, 13274082, 16072988, 58085,...","36448526, 36448541",Six children leave the comfort of their homes....
2,25574766,Il ladro di nebbia,https://www.goodreads.com//book/show/25574766-...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,Lavinia Petti,https://www.goodreads.com/author/show/14006459...,219,36,412.0,May 21st 2015,Longanesi,NaN,"Fantasy 8, Fiction 3",NaN,NaN,B00WX990QQ,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/1?...,https://www.goodreads.com//book_link/follow/8?...,"9707981, 9707981, 52331839, 25663617, 22027260...",NaN,Antonio M. Fonte Ã¨ uno scrittore di enorme su...
3,133411,"Gods, Graves and Scholars: The Story of Archae...",https://www.goodreads.com//book/show/133411.Go...,NaN,https://i.gr-assets.com/images/S/compressed.ph...,"C.W. Ceram, Sophie Wilkins, E.B. Garside",https://www.goodreads.com/author/show/4651612....,1834,145,536.0,1949,Vintage,"GÃ¶tter, GrÃ¤ber und Gelehrte: Roman der ArchÃ...","History 241, Nonfiction 149, History-Archaeolo...",0394743199,9.780395e+12,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"935992, 935992, 23487060, 189589, 883503, 9611...",NaN,C.W. Ceram visualized archeology as a wonderfu...
4,64279,Heaven and Hell,https://www.goodreads.com//book/show/64279.Hea...,(North and South #3),https://i.gr-assets.com/images/S/compressed.ph...,John Jakes,https://www.goodreads.com/author/show/5768.Joh...,10369,222,781.0,1987,Signet,Heaven and Hell,"Historical-Historical Fiction 459, Fiction 194...",0451200837,9.780451e+12,NaN,NaN,NaN,NaN,https://www.goodreads.com//book_link/follow/17...,https://www.goodreads.com//book_link/follow/8?...,"73062, 73062, 8716, 65959, 891922, 1503274, 16...","159178, 159177",The last days of the Civil War bring no peace ...


### Quick look on target feature column 

In [6]:
# import label column (average_rating)
Y = pd.read_csv("y_train.csv")
Y.head()

,id,average_rating
0,2533958,3.62
1,16071790,3.80
2,13825165,3.55
3,18061,4.24
4,3766428,4.33


# Describe the Datasets

In [7]:
print (f"Train set  has {X.shape[0]} rows and {X.shape[1]} columns")
print (f"Test set  has {X_test.shape[0]} rows and {X.shape[1]} columns")
print (f"Target feature set has {Y.shape[0]} rows")

Train set  has 41759 rows and 25 columns
Test set  has 10440 rows and 25 columns
Target feature set has 41759 rows


- It is obvious that two dataset train and set has the same numbers of columns 

- Target rows are equal to training data rows. 

## X_train Dataset

In [8]:
# training dataset info
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41759 entries, 0 to 41758
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      41759 non-null  int64  
 1   title                   41759 non-null  object 
 2   link                    41759 non-null  object 
 3   series                  18647 non-null  object 
 4   cover_link              41273 non-null  object 
 5   author                  41759 non-null  object 
 6   author_link             41759 non-null  object 
 7   rating_count            41759 non-null  int64  
 8   review_count            41759 non-null  int64  
 9   number_of_pages         39886 non-null  float64
 10  date_published          41071 non-null  object 
 11  publisher               38866 non-null  object 
 12  original_title          31463 non-null  object 
 13  genre_and_votes         39521 non-null  object 
 14  isbn                    32284 non-null

## Brief Descriptions: 

- There are four numerical data columns including int64 and float64 datatypes

- The rest is all object datatype including string values

In [9]:
# describe the columsn with object datatype
X.describe(include='object').T

,count,unique,top,freq
title,41759,40047,Legacy,11
link,41759,41759,https://www.goodreads.com//book/show/11549777-...,1
series,18647,18197,(Sherlock Holmes),12
cover_link,41273,41273,https://i.gr-assets.com/images/S/compressed.ph...,1
author,41759,23482,Nora Roberts,65
author_link,41759,19803,https://www.goodreads.com/author/show/3389.Ste...,82
date_published,41071,8299,2009,371
publisher,38866,9514,Vintage,457
original_title,31463,30516,Legacy,8
genre_and_votes,39521,37957,Fiction 1user,70


- It can be seen that there are large amounts of unique values for each object datatypes. 

In [10]:
# describe all numerical data columns 
X.describe().T

,count,mean,std,min,25%,50%,75%,max
id,41759.0,1.012620e+07,1.162310e+07,1.0,363987.5,6365996.0,17373246.0,54442989.0
rating_count,41759.0,1.893641e+04,1.204193e+05,0.0,346.0,2295.0,9271.0,6801077.0
review_count,41759.0,1.011812e+03,4.129329e+03,0.0,32.0,164.0,626.5,169511.0
number_of_pages,39886.0,3.293661e+02,2.509911e+02,0.0,211.0,304.0,393.0,11095.0


- min of three columns including "rating_count", "review_count" and "num_of_pages' have values 0. These could be considered as impossible values. 

- This should be handed in next steps. 

# X_test Dataset

#### With the same flows as training data 

In [11]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10440 entries, 0 to 10439
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      10440 non-null  int64  
 1   title                   10440 non-null  object 
 2   link                    10440 non-null  object 
 3   series                  4672 non-null   object 
 4   cover_link              10319 non-null  object 
 5   author                  10440 non-null  object 
 6   author_link             10440 non-null  object 
 7   rating_count            10440 non-null  int64  
 8   review_count            10440 non-null  int64  
 9   number_of_pages         9983 non-null   float64
 10  date_published          10268 non-null  object 
 11  publisher               9697 non-null   object 
 12  original_title          7787 non-null   object 
 13  genre_and_votes         9838 non-null   object 
 14  isbn                    8032 non-null 

- In training dataset, 'isbn13' is object values. But testing dataset shows that it is float64 datatypes. 

In [12]:
# cast 'isbn13' into object values
isbn13_float = X_test.isbn13
isbn13_float = X_test.isbn13.astype('object')

In [13]:
# review X_test info
X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10440 entries, 0 to 10439
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      10440 non-null  int64  
 1   title                   10440 non-null  object 
 2   link                    10440 non-null  object 
 3   series                  4672 non-null   object 
 4   cover_link              10319 non-null  object 
 5   author                  10440 non-null  object 
 6   author_link             10440 non-null  object 
 7   rating_count            10440 non-null  int64  
 8   review_count            10440 non-null  int64  
 9   number_of_pages         9983 non-null   float64
 10  date_published          10268 non-null  object 
 11  publisher               9697 non-null   object 
 12  original_title          7787 non-null   object 
 13  genre_and_votes         9838 non-null   object 
 14  isbn                    8032 non-null 

In [14]:
X_test.describe(include='object').T

,count,unique,top,freq
title,10440,10295,Selected Poems,4
link,10440,10440,https://www.goodreads.com//book/show/6580510-b...,1
series,4672,4627,(Star Wars Legends),5
cover_link,10319,10319,https://i.gr-assets.com/images/S/compressed.ph...,1
author,10440,7880,Nora Roberts,22
author_link,10440,7157,https://www.goodreads.com/author/show/625.Nora...,25
date_published,10268,3958,2009,91
publisher,9697,3648,Vintage,115
original_title,7787,7713,Hide and Seek,3
genre_and_votes,9838,9605,Fiction 1user,19


In [15]:
X_test.describe().T

,count,mean,std,min,25%,50%,75%,max
id,10440.0,1.026734e+07,1.172926e+07,6.000000e+00,3.583035e+05,6.441912e+06,1.741248e+07,5.398436e+07
rating_count,10440.0,1.862243e+04,9.869185e+04,0.000000e+00,3.210000e+02,2.285000e+03,9.400250e+03,2.905122e+06
review_count,10440.0,1.017657e+03,3.742072e+03,0.000000e+00,3.000000e+01,1.590000e+02,6.072500e+02,1.025470e+05
number_of_pages,9983.0,3.272512e+02,2.598594e+02,0.000000e+00,2.090000e+02,3.040000e+02,3.900000e+02,1.477700e+04
isbn13,7847.0,9.773283e+12,2.722149e+11,9.780616e+09,9.780385e+12,9.780747e+12,9.781479e+12,9.978069e+12


# Y_train Dataset

In [16]:
Y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41759 entries, 0 to 41758
Data columns (total 2 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              41759 non-null  int64  
 1   average_rating  41759 non-null  float64
dtypes: float64(1), int64(1)
memory usage: 652.6 KB


### I will drop 'id' column in this dataset in order to make Y as target feature with new dataset named Y_new


In [17]:
# drop 'id' to use target variable 'average_rating'
Y_target = Y.drop('id', axis=1)

Y_target

,average_rating
0,3.62
1,3.80
2,3.55
3,4.24
4,4.33
...,...
41754,4.31
41755,4.10
41756,4.31
41757,4.45


# Checking Missing Values

In [18]:
## a fucntion to return total percentage of the missing values

## reference here
def missing_percentage(df):
    total = df.isnull().sum().sort_values(ascending = False)
    
    percent = round(total / len(df)*100,2)
    
    return pd.concat([total, percent], axis=1, keys=['Total','Percent'])

### Missing values on train 

In [19]:
missing_percentage(X)

,Total,Percent
asin,37605,90.05
awards,33256,79.64
settings,32473,77.76
characters,30782,73.71
books_in_series,24096,57.70
series,23112,55.35
original_title,10296,24.66
isbn13,10099,24.18
isbn,9475,22.69
worldcat_redirect_link,3203,7.67


### Missing values on test 

In [20]:
missing_percentage(X_test)

,Total,Percent
asin,9358,89.64
awards,8317,79.66
settings,8211,78.65
characters,7722,73.97
books_in_series,6025,57.71
series,5768,55.25
original_title,2653,25.41
isbn13,2593,24.84
isbn,2408,23.07
recommended_books,843,8.07


In [21]:
missing_percentage(Y_target)

,Total,Percent
average_rating,0,0.0


# Imputing Missing Values

In [22]:
# Combine both train and test dataset to do works consistently
# all_data = pd.concat((X, X_test)).reset_index(drop=True)

# all_data.shape

There are missing values for both numerical and object datatypes. Thus, I do imputing by 2 different datatypes.

In [23]:
# imputing missing object data
missing_val_cate = ['asin',
                    'awards',
                    'settings',
                    "characters",
                    'books_in_series',
                    'series',
                    'original_title',
                    'isbn13',
                    'isbn',
                    'recommended_books',
                    'worldcat_redirect_link',
                    'publisher',
                    'genre_and_votes',
                    'description',
                    'date_published',
                    'cover_link'
                   ]
def imputing_na(df):
    for i in missing_val_cate:
        df[i] = df[i].fillna('None')

In [24]:
#creat a copy of X dataset
X_new = X.copy()

In [25]:
#imputing missing values on new copy
imputing_na(X_new)


In [26]:
X_test_new = X_test.copy()

imputing_na(X_test_new)

In [27]:
# imputing missing numerical data 

# Only 'number_of_pages' has missing values
X_new['number_of_pages'] = X['number_of_pages'].fillna(0)

In [28]:
X_test_new['number_of_pages'] = X_test_new['number_of_pages'].fillna(0)

In [29]:
missing_percentage(X_new)

,Total,Percent
description,0,0.0
publisher,0,0.0
title,0,0.0
link,0,0.0
series,0,0.0
cover_link,0,0.0
author,0,0.0
author_link,0,0.0
rating_count,0,0.0
review_count,0,0.0


In [30]:
missing_percentage(X_test_new)

,Total,Percent
description,0,0.0
publisher,0,0.0
title,0,0.0
link,0,0.0
series,0,0.0
cover_link,0,0.0
author,0,0.0
author_link,0,0.0
rating_count,0,0.0
review_count,0,0.0


#### Review object column descriptions.

In [31]:
X.describe(include='object').T

,count,unique,top,freq
title,41759,40047,Legacy,11
link,41759,41759,https://www.goodreads.com//book/show/11549777-...,1
series,18647,18197,(Sherlock Holmes),12
cover_link,41273,41273,https://i.gr-assets.com/images/S/compressed.ph...,1
author,41759,23482,Nora Roberts,65
author_link,41759,19803,https://www.goodreads.com/author/show/3389.Ste...,82
date_published,41071,8299,2009,371
publisher,38866,9514,Vintage,457
original_title,31463,30516,Legacy,8
genre_and_votes,39521,37957,Fiction 1user,70


Obviously it can be infered for some details in all data by unique values: 

 - The top of series is 'Star Wars Legends' with the count of 13
 - Nora Roberts (count of 65) is the author having the most books
 - The publiser has the most book is 'Vintage' (count of 572)
 - 'United States Of America" is the most 'settings'
 - The most 'awards' collected is 'Goodreads Choice Award Nominee for Romance (2014)'

## Assumptions: 
    
These columns are identified code of books. This will not give valuable explorations. Therefore, I will remove them.

- isbn: International Standard Book Number
- isbn13: 13-digit ISBN
- asin: Amazon Standard Identification Number

In addition, several "link" features can be dropped due to pointless insights:

- link
- amazon_redirect_link
- worldcat_redirect_link
- author_link
- cover_link

In [32]:
# separate all_data into train and test
# X_train, X_test = train_test_split(all_data, test_size=0.2, random_state=0)
# print(len(X_train))


In [33]:
X_train.isna().sum()

NameError: name 'X_train' is not defined

In [ ]:
import category_encoders as ce

In [ ]:

series_count = all_data['series'].value_counts()

series_count

In [ ]:
encoder = ce.target_encoder.TargetEncoder(cols=['series'])

encoder.fit(X_train['series'], Y_target)

In [ ]:
series = X_train['series']

In [ ]:
series = encoder.transform(series)

In [ ]:
series = series.round()
series

In [ ]:
# plt.figure(figsize=(20,15))
# sns.barplot(series_count.values, series_count.index, alpha=0.9)

# plt.show()

In [ ]:
X_train_demo = X_train.copy()

In [ ]:
X_train_demo.series = X_train_demo.series.astype('category')
X_train_demo.info()

In [ ]:
# lower, tokenize and stem
def text_tokenize(text):
    text = text.lower()
    word_tokens = nltk.word_tokenize(text) 
    stop_words = set(stopwords.words('english'))
    stop_words.update(['.', ',', '"', "'", ':', ';', '(', ')', '[', ']', '{', '}'])
    filtered_sentence = [w for w in word_tokens if not w in stop_words]  
    stems = []
    for item in filtered_sentence:
        stems.append(PorterStemmer().stem(item))
    return stems

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(tokenizer=text_tokenize)

In [ ]:
#demo dataset
demo = all_data.head(10)
demo

In [ ]:
demo = vectorizer.fit_transform(demo['description'])

In [ ]:
demo.shape

In [ ]:
demo = demo.toarray()

In [ ]:
column_list = vectorizer.get_feature_names()

In [ ]:
demo = pd.DataFrame(demo, columns=column_list)

In [ ]:
demo

In [ ]:
# labelEncode = preprocessing.LabelEncoder()

# all_data['series'] = labelEncode.fit_transform(all_data['series'])

In [ ]:
#view unique entries 

def unique (data,col):
    print(str(col)+': ', data[col].unique(), '\n'*2, " Len:",len(data[col].unique()))

In [ ]:
unique(all_data, 'series')

In [ ]:
unique(all_data, 'settings')

# demo feature select

In [ ]:
# final_fea = pd.get_dummies(X_new).reset_index(drop=True)
pd.options.display.max_columns = None

final_feature = pd.get_dummies(all_data, columns=['settings'])
final_feature.head()

# Task 1.2: Data Exploration 

# Overall Observations: 

* Two main types of features are object (contain string values) and numeric (int64 & float64).
* There are some missing values in feature shortlists. 

I am going to emphasize on **average_rating** scrapped to Y as target feature in this notebook. Thererfore,  

In [ ]:
def plotting_3_chart(df, feature):
    ## Importing seaborn, matplotlab and scipy modules. 
    import seaborn as sns
    import matplotlib.pyplot as plt
    import matplotlib.gridspec as gridspec
    from scipy import stats
    import matplotlib.style as style
    style.use('fivethirtyeight')

    ## Creating a customized chart. and giving in figsize and everything. 
    fig = plt.figure(constrained_layout=True, figsize=(12,8))
    ## creating a grid of 3 cols and 3 rows. 
    grid = gridspec.GridSpec(ncols=3, nrows=3, figure=fig)
    #gs = fig3.add_gridspec(3, 3)

    ## Customizing the histogram grid. 
    ax1 = fig.add_subplot(grid[0, :2])
    ## Set the title. 
    ax1.set_title('Histogram')
    ## plot the histogram. 
    sns.distplot(df.loc[:,feature], norm_hist=True, ax = ax1)

    # customizing the QQ_plot. 
    ax2 = fig.add_subplot(grid[1, :2])
    ## Set the title. 
    ax2.set_title('QQ_plot')
    ## Plotting the QQ_Plot. 
    stats.probplot(df.loc[:,feature], plot = ax2)

    ## Customizing the Box Plot. 
    ax3 = fig.add_subplot(grid[:, 2])
    ## Set title. 
    ax3.set_title('Box Plot')
    ## Plotting the box plot. 
    sns.boxplot(df.loc[:,feature], orient='v', ax = ax3 );
    
plotting_3_chart(Y, 'average_rating')

In [ ]:
print("Skewness: " + str(Y['average_rating'].skew()))
print("Kurtosis: " + str(Y['average_rating'].kurt()))

In [ ]:
Y['average_rating'] = np.log1p(Y["average_rating"])

plotting_3_chart(Y, 'average_rating')

# Multiple Linear

In [ ]:
X_mul = np.array(X[['review_count','rating_count']])

In [ ]:
X_mul

In [ ]:
Y_one = np.array(Y['average_rating']) 
Y_one 

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_mul, Y, test_size=0.2, random_state=0)

In [ ]:
np.random.seed(0)

In [ ]:
X_train

In [ ]:
model = linear_model.LinearRegression()
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_val)

In [ ]:
print('Root Mean Squared Error (RMSE): %.2f'
     % metrics.mean_squared_error(y_val,y_pred))

print('Mean Absolute Error (MAE): %.2f'
      % metrics.mean_absolute_error(y_val, y_pred))

# The coefficient of determination: 1 is perfect prediction
# Accuracy of the model 
print('Coefficient of determination (R2): %.2f'
      % metrics.r2_score(y_val, y_pred))

In [ ]:
X_new.hist(figsize=(20,15))
plt.show()

In [ ]:
Y1 = Y[['average_rating']]
# correlation analysis
df = pd.concat([X,Y1],axis=1)

corr_matrix = df.corr()
corr_matrix.style.background_gradient(cmap='Greens')

In [ ]:
(df.corr()**2)['average_rating'].sort_values(ascending=False)[1:]

In [ ]:
df_demo = df[df.average_rating >= 1.4]

In [ ]:
df_demo['average_rating'].hist(figsize=(20,15))
plt.show()

In [ ]:
print("Skewness: "+ str(df_demo['average_rating'].skew()))

### Note: Need to clear outliers for more neutral distributed

In [ ]:
X_demo = X_new[X_new.review_count <= 30]

In [ ]:
print("Skewness: "+ str(X_demo['review_count'].skew()))

In [ ]:
# sns.displot(X_new,x = 'rating_count',height=20)
X_demo.hist(figsize=(20,15))

In [ ]:
df1 = df[['title','author', 'average_rating']]

In [ ]:
demo = df1.sort_values('average_rating', ascending=False).set_index('title')
demo[:10]

In [ ]:
demo_au = df1.sort_values('average_rating', ascending=False).set_index('author')
demo_au[:10]

In [ ]:
X_new.head(3)

# Textual Data draft

In [ ]:
from empath import Empath
lexicon = Empath()